In [1]:
import os
import sys

In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

In [3]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils import region_formatter

In [4]:
plt.rcParams["figure.figsize"] = [7.50, 5.50]
plt.rcParams["figure.autolayout"] = True

# Utenti in carico secondo la sostanza d'abuso primaria - Oppiacei (Anno 2022)

## Descrizione 

*"Il data-set contiene i dati relativi agli utenti in carico ai servizi pubblici per le dipendenze patologiche da sostanze nell'anno di riferimento.*

*Le informazioni contenute nel data-set sono elaborate in riferimento ai dati che le Regioni trasmettono al Ministero della salute, ai sensi del decreto del Ministro della salute 11 giugno 2010.*

*Per ciascun servizio sono riportati i dati relativi agli utenti per categoria di sostanza d'abuso."* (*fonte Ministero della Salute*)


[Link Dataset](https://www.dati.salute.gov.it/dati/dettaglioDataset.jsp?menu=dati&idPag=190) 



# prepare dataframe

In [5]:
df = pd.read_csv(
    '../datasets/C_17_dataset_190_0_upFile.csv',
    delimiter=";",
    skiprows=3,
    keep_default_na=False)

In [6]:
df.loc[df.Utenti == '']

,Anno,Codice regione,Descrizione Regione,Codice ASL,ASL,SERD,Categoria Sostanza,Utenti
40,2022,10,PIEMONTE,210,CN1,A01,OPPIACEI,
44,2022,10,PIEMONTE,210,CN1,S06,OPPIACEI,
62,2022,10,PIEMONTE,301,ASL CITTA' DI TORINO,A02,OPPIACEI,
97,2022,30,LOMBARDIA,706,ASST RHODENSE,S01,OPPIACEI,
277,2022,90,TOSCANA,201,AZIENDA USL TOSCANA CENTRO,S24,OPPIACEI,
...,...,...,...,...,...,...,...,...
2773,2022,200,SARDEGNA,201,AZIENDA PER LA TUTELA DELLA SALUTE,S05,ALTRE SOSTANZE,
2774,2022,200,SARDEGNA,201,AZIENDA PER LA TUTELA DELLA SALUTE,S06,ALTRE SOSTANZE,
2775,2022,200,SARDEGNA,201,AZIENDA PER LA TUTELA DELLA SALUTE,S12,ALTRE SOSTANZE,
2776,2022,200,SARDEGNA,201,AZIENDA PER LA TUTELA DELLA SALUTE,S14,ALTRE SOSTANZE,


In [7]:
df = df.replace(r'^\s*$', np.nan, regex=True)

In [8]:
df.loc[df.Utenti == '']

,Anno,Codice regione,Descrizione Regione,Codice ASL,ASL,SERD,Categoria Sostanza,Utenti


In [9]:
df.Utenti = df.Utenti.astype(float)

In [10]:
%run ../dataframes/serd.ipynb

In [11]:
df_oppiacei_agg_serd = pd.read_pickle("df_serd_regions_agg.pkl")

In [12]:
df_oppiacei_agg_serd

,reg_name,serd_count
0,abruzzo,11
1,basilicata,4
2,calabria,16
3,campania,43
4,emilia romagna,12
5,friuli venezia giulia,13
6,lazio,16
7,liguria,6
8,lombardia,22
9,marche,14


# Prepare geodataframe

In [13]:
if not os.path.exists("regions_df.pkl"):
    %run ../dataframes/regions_gdf.ipynb -i

In [14]:
regions_df = pd.read_pickle("regions_df.pkl")

In [15]:
regions_df

,reg_name,reg_istat_code_num,reg_istat_code,geometry,zone
0,piemonte,1,01,"POLYGON ((7.13605 45.27996, 7.13601 45.28040, ...",Nord
1,valle d'aosta,2,02,"POLYGON ((7.73455 45.92365, 7.73590 45.92744, ...",Nord
2,lombardia,3,03,"MULTIPOLYGON (((8.57964 45.75172, 8.57532 45.7...",Nord
3,trentino alto adige,4,04,"POLYGON ((12.01396 46.55091, 12.01376 46.55077...",Nord
4,veneto,5,05,"POLYGON ((10.63538 45.53543, 10.63228 45.56846...",Nord
5,friuli venezia giulia,6,06,"MULTIPOLYGON (((13.51691 46.38301, 13.51750 46...",Nord
6,liguria,7,07,"MULTIPOLYGON (((7.55166 43.89429, 7.56264 43.9...",Nord
7,emilia romagna,8,08,"MULTIPOLYGON (((9.33258 44.73504, 9.33167 44.7...",Centro
8,toscana,9,09,"MULTIPOLYGON (((9.89786 44.16359, 9.89451 44.1...",Centro
9,umbria,10,10,"MULTIPOLYGON (((13.15849 42.65425, 13.15764 42...",Centro


In [16]:
df["Descrizione Regione"].unique()

array(['piemonte', "valle d'aosta", 'lombardia', 'trentino alto adige',
       'veneto', 'friuli venezia giulia', 'liguria', 'emilia romagna',
       'toscana', 'umbria', 'marche', 'lazio', 'abruzzo', 'molise',
       'campania', 'puglia', 'basilicata', 'calabria', 'sicilia',
       'sardegna'], dtype=object)

In [17]:
region_formatter = region_formatter.RegionFormatter(df)

In [18]:
region_formatter.format("Descrizione Regione")

In [19]:
df["Descrizione Regione"].unique()

array(['piemonte', "valle d'aosta", 'lombardia', 'trentino alto adige',
       'veneto', 'friuli venezia giulia', 'liguria', 'emilia romagna',
       'toscana', 'umbria', 'marche', 'lazio', 'abruzzo', 'molise',
       'campania', 'puglia', 'basilicata', 'calabria', 'sicilia',
       'sardegna'], dtype=object)

In [20]:
df = df.rename(columns={"Descrizione Regione": "reg_name"})

In [21]:
df

,Anno,Codice regione,reg_name,Codice ASL,ASL,SERD,Categoria Sostanza,Utenti
0,2022,10,piemonte,203,TO3,A12,OPPIACEI,1
1,2022,10,piemonte,203,TO3,B01,OPPIACEI,3
2,2022,10,piemonte,203,TO3,B03,OPPIACEI,40
3,2022,10,piemonte,203,TO3,S01,OPPIACEI,124
4,2022,10,piemonte,203,TO3,S02,OPPIACEI,104
...,...,...,...,...,...,...,...,...
2775,2022,200,sardegna,201,AZIENDA PER LA TUTELA DELLA SALUTE,S12,ALTRE SOSTANZE,
2776,2022,200,sardegna,201,AZIENDA PER LA TUTELA DELLA SALUTE,S14,ALTRE SOSTANZE,
2777,2022,200,sardegna,201,AZIENDA PER LA TUTELA DELLA SALUTE,S21,ALTRE SOSTANZE,
2778,2022,200,sardegna,302,ASL N. 2 DELLA GALLURA,S06,ALTRE SOSTANZE,1


In [22]:
df = df.drop(columns=["ASL", "SERD", "Anno", "Codice ASL", "Codice regione"])

# Opiates data 

In [23]:
df_oppiacei = df.loc[df["Categoria Sostanza"] == 'OPPIACEI', ]

In [24]:
df_oppiacei["Categoria Sostanza"].unique()

array(['OPPIACEI'], dtype=object)

In [25]:
df_oppiacei = df_oppiacei.drop(columns=["Categoria Sostanza", ])

In [26]:
df_oppiacei = df_oppiacei.groupby([
    'reg_name',
]).sum().reset_index()

In [27]:
df_oppiacei = df_oppiacei.rename(columns={"Utenti": "utenti_sum"})

In [28]:
df_oppiacei

,reg_name,utenti_sum
0,abruzzo,135127140218242274491109382241213
1,basilicata,8817711778343223
2,calabria,4171591095213465152279283133218144467319644
3,campania,1683301378075179169891143362431182861361371631...
4,emilia romagna,4963159688537011339792733379955658259837298613...
5,friuli venezia giulia,4327372710449275732344139683109
6,lazio,4714010110721624661676815216510017035641062581...
7,liguria,16769130118856135336284238217240151310255176
8,lombardia,2011053747263624745611910115290226483484411177...
9,marche,233107116183219524182752846631823533753


In [29]:
df_oppiacei = df_oppiacei.merge(df_oppiacei_agg_serd, on="reg_name")

In [30]:
df_oppiacei["utenti_std"] = df_oppiacei["utenti_sum"] / df_oppiacei["serd_count"]

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [ ]:
df_oppiacei

In [ ]:
gdf_oppiacei = regions_df.merge(df_oppiacei)

In [ ]:
gdf_oppiacei

In [ ]:
gdf_oppiacei_agg = gdf_oppiacei[["reg_name", 'utenti_sum', 'zone', 'serd_count']].groupby("reg_name").sum()

In [ ]:
region_colours = {
    "Nord": "#2a9d8f", 
    "Centro": "#e76f51",
    "Sud e Isole": "#e9c46a",
}

In [ ]:
gdf_oppiacei_agg_barplot = gdf_oppiacei_agg.sort_values(by=['utenti_sum']).plot.barh(
    y="utenti_sum",
    ylabel='Regioni',
    title="Totale utilizzatori di oppiacei in carico ai SerD per regione - Anno 2022",
    color=gdf_oppiacei_agg.sort_values(by=['utenti_sum'])['zone'].replace(region_colours)
).legend(
    [
        Patch(facecolor=region_colours['Nord']),
        Patch(facecolor=region_colours['Centro']),
        Patch(facecolor=region_colours['Sud e Isole']),
    ], ["Nord", "Centro", 'Sud e Isole']
)
plt.annotate('fonte: Ministero della salute', (0,0), (260, -30), xycoords='axes fraction', textcoords='offset points', va='top')

In [ ]:
gdf_oppiacei_agg_barplot.get_figure().savefig('output/gdf_oppiacei_agg_barplot.png')

In [ ]:
plot_oppiacei = gdf_oppiacei.plot(
    column="utenti_sum",
    legend=True,
    cmap='OrRd'
);
plt.title("Totale Utilizzatori di oppiacei in carico ai SerD per regione - Anno 2022")
plot_oppiacei.set_axis_off()
plt.annotate('fonte: Ministero della salute', (0,0), (260, -30), xycoords='axes fraction', textcoords='offset points', va='top')

In [ ]:
plot_oppiacei.get_figure().savefig('output/plot_oppiacei.png')

In [ ]:
gdf_oppiacei_agg_avg = gdf_oppiacei[["reg_name", 'utenti_std', 'zone']].groupby("reg_name").sum()

In [ ]:
gdf_oppiacei_agg_avg.sort_values(by=['utenti_std'])['zone']

In [ ]:
gdf_oppiacei_agg_avg_barplot = gdf_oppiacei_agg_avg.sort_values(by=['utenti_std']).plot.barh(
    y="utenti_std",
    ylabel='Regioni',
    title="Media utilizzatori di oppiacei per SerD per regione - Anno 2022",
    color=gdf_oppiacei_agg_avg.sort_values(by=['utenti_std'])['zone'].replace(region_colours)
).legend(
    [
        Patch(facecolor=region_colours['Nord']),
        Patch(facecolor=region_colours['Centro']),
        Patch(facecolor=region_colours['Sud e Isole']),
    ], ["Nord", "Centro", 'Sud e Isole']
)
plt.annotate('fonte: Ministero della salute', (0,0), (260, -30), xycoords='axes fraction', textcoords='offset points', va='top')

In [ ]:
gdf_oppiacei_agg_avg_barplot.get_figure().savefig('output/gdf_oppiacei_agg_avg_barplot.png')

In [ ]:
plot_oppiacei_avg = gdf_oppiacei.plot(
    column="utenti_std",
    legend=True,
    cmap='OrRd'
)
plt.title("Media Utilizzatori di Oppiacei per SerD per regione - Anno 2022")
plot_oppiacei_avg.set_axis_off()
plt.annotate('fonte: Ministero della salute', (0,0), (260, -30), xycoords='axes fraction', textcoords='offset points', va='top')

In [ ]:
plot_oppiacei_avg.get_figure().savefig('output/plot_oppiacei_avg.png')

In [ ]:
gdf_serd_agg_barplot = gdf_oppiacei_agg.sort_values(by=['serd_count']).plot.barh(
    y="serd_count",
    ylabel='Regioni',
    title="Totale SerD per regione - Anno 2022",
    color=gdf_oppiacei_agg.sort_values(by=['serd_count'])['zone'].replace(region_colours)
).legend(
    [
        Patch(facecolor=region_colours['Nord']),
        Patch(facecolor=region_colours['Centro']),
        Patch(facecolor=region_colours['Sud e Isole']),
    ], ["Nord", "Centro", 'Sud e Isole']
)
plt.annotate('fonte: Ministero della salute', (0,0), (260, -30), xycoords='axes fraction', textcoords='offset points', va='top')

In [ ]:
gdf_serd_agg_barplot.get_figure().savefig('output/gdf_serd_agg_barplot.png')

In [ ]:
plot_serd = gdf_oppiacei.plot(
    column="serd_count",
    legend=True,
    cmap='OrRd'
)
plt.title("Totale SerD per regione - Anno 2022")
plot_oppiacei_avg.set_axis_off()
plt.annotate('fonte: Ministero della salute', (0,0), (260, -30), xycoords='axes fraction', textcoords='offset points', va='top')

In [ ]:
plot_serd.get_figure().savefig('output/plot_serd.png')